# Открываем файлы предыдущей разметки
Преобразуем в формат, удобный для анализа структуры и поиска тегов

In [642]:
def openfile(name):
    with open(name, encoding='utf-8') as f:
        file = f.read()
        
    return file

In [643]:
files = []
for i in range(1, 50):
    print("файл", i, end=" -- ")
    name = "/Users/alinazabolotskaya/Desktop/annotated/" + str(i)+".xml"
    try:
        text = openfile(name)
        files.append(text)
    except FileNotFoundError:
        pass

файл 1 -- файл 2 -- файл 3 -- файл 4 -- файл 5 -- файл 6 -- файл 7 -- файл 8 -- файл 9 -- файл 10 -- файл 11 -- файл 12 -- файл 13 -- файл 14 -- файл 15 -- файл 16 -- файл 17 -- файл 18 -- файл 19 -- файл 20 -- файл 21 -- файл 22 -- файл 23 -- файл 24 -- файл 25 -- файл 26 -- файл 27 -- файл 28 -- файл 29 -- файл 30 -- файл 31 -- файл 32 -- файл 33 -- файл 34 -- файл 35 -- файл 36 -- файл 37 -- файл 38 -- файл 39 -- файл 40 -- файл 41 -- файл 42 -- файл 43 -- файл 44 -- файл 45 -- файл 46 -- файл 47 -- файл 48 -- файл 49 -- 

In [644]:
joined = ' '.join(files)

In [645]:
with open("/Users/alinazabolotskaya/Desktop/annotated/oldxmls1.xml", "w") as text_file:
    text_file.write("<texts>")
    text_file.write(joined)
    text_file.write("</texts>")

### Создадим вариант текстов только с японской разметкой

In [646]:
import re


In [647]:
joined = re.sub(r'<russian>[\s\S]*?<\/russian>', '', joined, re.DOTALL)
print(joined[:200])

<entry>
    <id>1</id>
    <url>
        
        <japanese>https://www.hokkoku.co.jp/subpage/HT20200205401.htm</japanese>
    </url>
    <title>
        
        <japanese>ロシア捕虜のベッドか　日露戦争時、金沢で収容　白山市で


# Поиск тегов в существующем XML
С помощью regex можно найти существующие теги и посмотреть, как их размечает JUMAN++. Например, личное местоимение あなた ("ты") размечается как Noun ("существительное")

In [648]:
import re

In [649]:
result = re.findall(r'<name>pos</name>[^,]*?<value>あなた</value>', joined, re.DOTALL)#[\s\S]*?
result

["<name>pos</name>\n                                <value>Noun</value>\n                            </attribute>\n                            <attribute>\n                                <name>category</name>\n                                <value>['People']</value>\n                            </attribute>\n                            <attribute>\n                                <name>romaji_reading</name>\n                                <value>anata</value>\n                            </attribute>\n                            <attribute>\n                                <name>lexeme_reading</name>\n                                <value>あなた</value>"]

In [650]:
for i in result[0].split("\n"):
    print(i)

<name>pos</name>
                                <value>Noun</value>
                            </attribute>
                            <attribute>
                                <name>category</name>
                                <value>['People']</value>
                            </attribute>
                            <attribute>
                                <name>romaji_reading</name>
                                <value>anata</value>
                            </attribute>
                            <attribute>
                                <name>lexeme_reading</name>
                                <value>あなた</value>


#  Чтение XML файла

In [651]:
import xml.etree.ElementTree as ET
import xml.dom.minidom 

In [652]:
def to_tree(name):
    
    tree = ET.parse(name)
    root = tree.getroot()
    return root

# Переразметка токенов

In [653]:
# Далее из XML файла вытаскивается нужная информация и происходит переразметка с суффиксами и префиксами

#Алгоритм таков:
#    1) Достаем айди предложения, его перевод на русский и всю информацию о токене
#    2) Создаем пустой список для записи морфем, которые должны объединиться в слово
#    3) Если список морфемами слова пустой, 
#        а) добавляем текущий токен в список морфем, 
#        б) часть речи токена записываем в переменную previous - предыдущая морфема
#    4) Если в списке что-то уже есть, проверяем, началось ли новое слово!
#        а) если текущая морфема не суффикс, и предыдущая не приставка:
#                    -> началось новое слово: -список морфем добавляется к списку слов, 
#                                             -список морфем снова становится пустым
#        в) если новое слово не началось, повторяем пункты 3а) и 3б)

In [654]:
def restructure(root):
    data = [] #список, в который мы будем вытаскивать нужную информацию о предложении: 
                        #айди, предложение на русском и японскую разметку в виде словаря

    for child in root[3]:
        s_id = child[0].text
        s_rus = child[1][0].text
        wordlist = [] # сюда сложим [префиксы, основу и суффиксы] списком в виде объектов деревьев и отправим в функцию на аннотацию
        morphlist = [] # здесь соберем слово
        for token in child[2][1]:
            if morphlist == []:
                try:
                    for attr in token[5]:
                        if attr[0].text == 'pos':
                            previous = attr[1].text
                    morphlist.append(token)
                    #print(len(morphlist))
                except IndexError: #не морфемные токены, например, знаки пунктуации
                        #for t in token:
                        #print(t.tag,t.text)
                    wordlist.append([token])
            else:
                has_a = 0
                for t in token:
                    if t.tag == 'attributes':
                        has_a = 1
                        for attr in t:
                            if attr[0].text == 'pos':
                                #print(attr[1].text)
                                if attr[1].text != 'Suffix' and previous != 'Prefix':
                                    #print("Новое слово")
                                    wordlist.append(morphlist)
                                    morphlist = []
                                else:
                                    pass#print(attr[1].text, previous)
                                previous = attr[1].text
                        morphlist.append(token)
                        #print(len(morphlist))
                if has_a == 0:
                        #wordlist.append(morphlist)
                    wordlist.append([token])
                    morphlist = []

                
        data.append((s_id, s_rus, wordlist))
    c = 0
    for i in data:
        for j in i[2]:
            if len(j) > 1:
                c += 1
    print("Количество переразмеченных слов в этом тексте:", c)
    return data


# Промежуточный формат, удобный для аннотирования
#### Теперь запишем в словарь теги из объектов дерева и отправим токены на аннотацию:

In [655]:
def get_tags(data):
    all_data = []
    for se in data:
        full_pre = []
        trouble = se[2]
        pre_wordlist = []
        for token in trouble:
            pre_dict = {}
            if len(token) == 1: #для токенов без аффиксов функция аннотации отлична от функции с аффиксами, так как требуется усложненный глоссинг 
                for attr in token[0]:
                    if attr.tag not in ["attributes", "extraAttributes"]:
                        if attr.tag not in pre_dict:
                            pre_dict.update({attr.tag: attr.text})
                        else:
                            print(attr.tag, "повторяется!")
                    elif attr.tag == "attributes":
                        for a in attr:
                            if a[0].text not in pre_dict:
                                pre_dict.update({a[0].text: a[1].text})
                            else:
                                print(a[0].text, "повторяется!")

                    elif attr.tag == "extraAttributes":
                        extra = []
                        for a in attr:
                            extra.append(a.text)
                        pre_dict.update({attr.tag: extra})

                    else:
                        print("!!!!unknown attrib", attr.tag)
                ann_dict = ana(pre_dict)
                pre_wordlist.append(ann_dict)
            else:
                pass 
        full_pre.append(pre_wordlist)
        ready = (se[0], se[1], full_pre)
        all_data.append(ready)
    return all_data


#### Как мы видим, теги не повторяются! Поэтому можно использовать формат словаря

# Реализация разметки. Приписывание тегов. Создание слоев

#### Переразметка местоимений:

In [656]:
pron = "You, あちこち, あちら, あなた, ある, あれ, いずこ, いずれ, いつ, おまえ, おめー, お前, か, かれ, ここ, こちら, こっち, これ, こんな, そこ, そちら, そらあ, それ, そんな, どこ, どちら, どなた, どれ, どんな, なに, なん, ぼく, わしゃ, わたし, われわれ, オタク, キミ, ボク, ヲタク, 何, 何処, 何時, 余, 俺, 僕, 君, 己, 彼, 彼女, 彼方, 我, 我々, 私, 誰, 貴様".split(", ")

In [657]:
def clean_tr(text): #в дальнейшем возможно подключение mystem, nltk
    text = re.sub('<i>|</i>|<b>|</b>|<br>|</br>', ' ', text)
    text = re.sub('<span class="gtat">', '(', text)
    text = re.sub('</span>', ')', text)
    if text.startswith("('"): text = text[2:]
    if text.endswith("',)"): text = text[:-3]
    return text.strip()

In [671]:
def ana(dic):
    text, gr, has_translit, gloss, lex, transl, sem, translit, hiragana, lex_translit, hiragana_wf, ortho, morphon = '','','','','','','','','','','','',''
    text = dic["text"]
    if dic["text"] == "高岸寺":
        result = {"text": "高岸寺"}
    else:
        if text.isalpha() == True:
            if 'lexeme' in dic:
                lex = ''
                lex = dic['lexeme']

                
        #ЧАСТИ РЕЧИ
            gr_tags =[]              # -- CHANGES
            if 'pos' in dic:
                pos = ''
                if dic['pos'] == 'Verb':
                    if 'type' in dic and dic['type'] == 'Auxiliary':
                        pos = 'AUX'
                    else: pos = 'V'

                elif dic['pos'] == 'Adjective':
                    if 'type' in dic:
                        if dic['type'] == 'Na': pos = 'A,ADNOM1'
                        elif dic['type'] == 'I': pos = 'A,ADPRAEDIC'
                    else: pos = 'A,ADNOM2' 
                elif dic['pos'] == 'Noun':
                    if dic['text'] in pron:
                        pos = 'PRO'
                    else: pos = 'S'
                else: pos = pos_map[dic['pos']]
                if pos:
                    gr_tags.append(pos)              # -- CHANGES




        #ГЛАГОЛЬНОЕ СЛОВОИЗМЕНЕНИЕ
                
                if pos == 'V':
                    if 'row' in dic: gr_tags.append(dic['row'][0].lower())
                    if 'stem_type' in dic: gr_tags.append(verb_map[dic['stem_type']])
                    else: gr_tags.append("3")
                    if 'form1' in dic: gr_tags.append(verb_map[dic['form1']])
                    if 'transitivity' in dic: gr_tags.append(verb_map[dic['transitivity']])
                    if 'form2' in dic: gr_tags.append(verb_map[dic['form2']])
                        
        #СЛОВОИЗМЕНЕНИЕ ПРИЛАГАТЕЛЬНЫХ 
                if pos in ["A,ADPRAEDIC","A,ADNOM1","A,ADNOM2"]:
                    if 'form1' in dic: gr_tags.append(adj_map[dic['form1']])
                    if 'transitivity' in dic: gr_tags.append(adj_map[dic['transitivity']])
                    if 'form2' in dic: gr_tags.append(adj_map[dic['form2']])

                gr = ",".join(gr_tags)
                
                
        #ТЕГИ ЧАСТИЦ
                if pos == 'PART':
                    if 'type' in dic and dic['type'] == 'Case_marking': 
                        gr_tags.append("case")
                        if dic['romaji_reading'] == "ga": gr_tags.append("nom")
                        elif dic['romaji_reading'] == "no": gr_tags.append("gen")
                        elif dic['romaji_reading'] == "ni": gr_tags.append("dat")
                        elif dic['romaji_reading'] == "wo": gr_tags.append("acc")
                        elif dic['romaji_reading'] == "e": gr_tags.append("lat")
                        elif dic['romaji_reading'] == "kara": gr_tags.append("abl")
                        elif dic['romaji_reading'] == "de": gr_tags.append("ins")
                        elif dic['romaji_reading'] == "made": gr_tags.append("ill")
                        elif dic['romaji_reading'] == "yori": gr_tags.append("eq")
                    elif 'type' in dic and dic['type'] == 'Adverbial': 
                        gr_tags.append("adv")
                        if 'romaji_reading' == "wa": gr_tags.append("top")
                        elif 'romaji_reading' == "to": gr_tags.append("com")
                    elif 'type' in dic and dic['type'] == 'Conjunctive':
                        gr_tags.append("conj")
                    elif 'type' in dic and dic['type'] == 'Sentence_ending':
                        gr_tags.append("sfin")
                    elif 'type' in dic and dic['type'] not in ['Conjunctive', 'Case_marking', 'Adverbial']:
                        print(dic['type'])
                gr = ",".join(gr_tags)    
                            

        #СЛОЙ ПЕРЕВОДА
                if 'translation' in dic and type(dic['translation']) == str: 
                    transl = clean_tr(dic['translation'])
                    if transl == 'перевод отсутствует':      #-----CHANGES
                        transl = ''
                    elif transl.startswith('перевод неоднозначен'):
                        transl = transl[20:]
                    
                    
        #СЛОЙ СЕМАНТИКИ
            sem = ''

            if "domain" in dic:
                domain = dic['domain'][2:-2].split("', '")
                
                for d in domain:
                    if sem != '':
                        sem += ','
                    sem = sem + sem_map[d]

            if "category" in dic:
                cat = dic['category'][2:-2].split("', '")
                
                for c in cat:
                    if sem != '':
                        sem += ','
                    sem = sem + sem_map[c]


        #СЛОИ ТРАНСЛИТЕРАЦИИ         #-----CHANGES
            translit, hiragana, lex_translit, hiragana_wf, has_translit = '', '', '', '', ''
            if 'romanized_lexeme_reading' in dic: 
                lex_translit = clean_tr(dic['romanized_lexeme_reading'])
                has_translit = 'true'
            if 'lexeme_reading' in dic: hiragana = dic['lexeme_reading']
            if 'romaji_reading' in dic: 
                translit = clean_tr(dic['romaji_reading'])
                has_translit = 'true'
            if 'hiragana_wf' in dic: hiragana_wf = dic['reading']
          
        
        #ДРУГИЕ СЛОИ 
            if 'extraAttributes' in dic: 
                extras = dic['extraAttributes']
                ortho = []
                if "Kanji" in extras: ortho.append('kanji')
                if "Hiragana" in extras: ortho.append('hiragana')
                if "Katakana" in extras: ortho.append('katakana')
                if "Latin_letters" in extras: ortho.append('romaji')
                if "Digits" in extras: ortho.append('digit')
                ortho = ','.join(ortho)

                morphon = []
                if "Euphonic_change" in extras: morphon.append('euphonic')
                if "Nasalization_change" in extras: morphon.append('nasalisation')
                morphon = ','.join(morphon)

            result = {
    "text": text,
    #"gloss": gloss,
    "lex": lex,
    "gr": gr,
    "transl": transl,
    "sem": sem,
    "translit": translit,                                                            #---- CHANGES
    'has_translit': has_translit,
    'lex_translit': lex_translit,      
    "hiragana": hiragana,
    "hiragana_wf": hiragana_wf,                                            #---- CHANGES
    "ortho": ortho,
    "morphon": morphon
                  }
        else:
            result = {'text': dic['text'], 'punct': True}
    return result
    #else: return {'text': dic['text'], 'punct': True}



In [659]:
result = re.findall(r"""<value>Particle</value>
                            </attribute>
                            <attribute>
                                <name>romaji_reading</name>
                                <value>[^,]*?</value>""", joined, re.DOTALL)#[\s\S]*?
ref = []
for r in result:
    if r not in ref:
        ref.append(r)


## Здесь описаны соответствия тегов

In [660]:
verb_map = {
    "Consonant": "1",
    "Vowel": "2",
    "Transitive": "tran",
    "Intransitive": "intr",
    "Ta": "pfv",
    "Basic": "inf",
    "Te": "cnv",
    "Volitional": "vol",
    "Assumptional": "hor",
    "Conjunctive": "conj",
    "Conditional": "cond",
    "Imperative": "imp",
    "Written": "wr",
    "Imperfective": "ipf",
    "Both": ""
}

In [661]:
adj_map = {
    "Ta": "pfv",
    "Transitive": "tran",
    "Intransitive": "intr",
    "Basic": "inf",
    "Te": "cnv",
    "Volitional": "vol",
    "Assumptional": "hor",
    "Conjunctive": "conj",
    "Conditional": "cond",
    "Imperative": "imp",
    "Written": "wr",
    "Imperfective": "ipf",
    "Both": ""
}

In [662]:
sem_map = {
"Culture and fine arts": "top:cult",
"Recreation": "top:recrea",
"Sports": "top:sports",
"Health and medicine": "top:health", 
"Home and family life": "top:family",
"Food and cooking": "top:cook",
"Transportation": "top:transp",
"Education and learning": "top:learn",
"Science and art": "top:science",
"Business": "top:business",
"Media": "top:media",
"Politics": "top:politics",
"Unspecified": "top:unspec",
"Organisations and associations": "t:orgz",
"Animals": "t:animal",
"Plants": "t:plant",
"Animals-Parts": "pt:part",
"Plants-Parts": "pt:part",
"Crafted-Food":"t:tool:food",
"Crafted-Clothes":"t:tool:cloth",
"Crafted-Vehicles":"t:tool:transp",
"Crafted-Money":"t:tool:money",
"Crafted-Other":"t:tool:other",
"Nature":"t:nature",
"Places-Facilities":"t:place:facil",
"Places-Facilities-Parts":"pt:part & pc:place:facil",
"Places-Nature":"t:place:nature",
"Places-Functional":"t:place:funct",
"Places-Other":"t:place:other",
"Abstract":"t:abstr",
"Phenomena-Nature":"t:phenom:nature",
"Phenomena-Life":"t:phenom:life",
"Actions":"t:action",
"Incidents":"t:incident",
"States":"t:state",
"Feelings":"t:psych",
"Systems and rules":"t:rules",
"Intellectual products":"t:ment",
"Power":"t:power",
"Abstract-Functional":"t:abstr:funct",
"Abstract-Other":"t:abstr:other",
"Shapes and figures":"t:physq:form",
"Colors":"t:color",
"Quantity":"t:quant",
"People":"t:people",
"Time":"t:time"
}

In [663]:
pos_map = {
    'Noun': 'S',
    'Verb': 'V',
    'Adjective': 'A',
    'Particle': 'PART',
    'Numeral': 'NUM',
    'Judgemental': 'AUX',
    'Adverb': 'ADV',
    'Demonstrative': 'PRO',
    'Conjunction': 'CONJ',
    'Interjection': 'INTJ',
    'Suffix': "SUFFIX!",
    'Unknown': "Unknown"
}

# Выгрузка разметки в XML файл формата НКРЯ

In [664]:
def to_file(sents, name): # list of tuples: (sentence id, sentence in russian, list of words as dicts with text and tags)

# create the file structure
    data = ET.Element('?xml version="1.0" encoding="utf-8"?')
    html = ET.SubElement(data, 'html')
    head = ET.SubElement(html, 'head')
    body = ET.SubElement(html, 'body')
    for se in sents:
        para = ET.SubElement(body, 'para')
        para.set('id', se[0])
        
        rus = ET.SubElement(para, 'se')
        rus.set('lang', 'rus')
        rus.text = se[1]
        
        jap = ET.SubElement(para, 'se')
        jap.set('lang', 'jpn')
        
        for word in se[2][0]: #se[2] - список словарей word с ключами text, lex и другие
            w = ET.SubElement(jap, 'w')
            #if word[has_translit] ==
            if "punct" not in word.keys():
                ana = ET.SubElement(w, 'ana')
                ana.text = word['text']
            
                for key in word.keys():
                    #print(key, word[key])
                    if key != 'text' and word[key] != "": 
                        
                        if key == 'has_translit' and word[key] == 'true':           # -------- CHANGES
                            w.set('has_translit', 'true')           # -------- CHANGES
                        else:
                            ana.set(key, word[key])
            else:
                w.text = word['text']
                    

    # create a new XML file with the results
    mydata = ET.tostring(data, encoding="unicode")
    
    with open(name, "w") as f:
        f.write(mydata)


In [672]:
exname = "/Users/alinazabolotskaya/Desktop/annotated/11.xml"
exnew_name = "/Users/alinazabolotskaya/Desktop/annotated/11_new.xml"
tree = to_tree(exname)
data = get_tags(restructure(tree))
to_file(data, exnew_name)

Количество переразмеченных слов в этом тексте: 37


# Переразметка всего корпуса

In [673]:
for i in range(1, 50):
    print(i)
    name = "/Users/alinazabolotskaya/Desktop/annotated/" + str(i)+".xml"
    new_name = "/Users/alinazabolotskaya/Desktop/annotated/" + str(i)+"_new.xml"
    try:
        tree = to_tree(name)
        data = get_tags(restructure(tree))
        to_file(data, new_name)
    except FileNotFoundError:
        pass

1
Количество переразмеченных слов в этом тексте: 42
2
3
Количество переразмеченных слов в этом тексте: 14
4
Количество переразмеченных слов в этом тексте: 48
5
Количество переразмеченных слов в этом тексте: 43
6
Количество переразмеченных слов в этом тексте: 106
7
Количество переразмеченных слов в этом тексте: 25
8
Количество переразмеченных слов в этом тексте: 63
9
Количество переразмеченных слов в этом тексте: 33
10
11
Количество переразмеченных слов в этом тексте: 37
12
Количество переразмеченных слов в этом тексте: 33
13
Количество переразмеченных слов в этом тексте: 28
14
Количество переразмеченных слов в этом тексте: 14
15
Количество переразмеченных слов в этом тексте: 35
16
Количество переразмеченных слов в этом тексте: 114
17
Количество переразмеченных слов в этом тексте: 111
attachment_form повторяется!
18
Количество переразмеченных слов в этом тексте: 51
19
Количество переразмеченных слов в этом тексте: 77
20
Количество переразмеченных слов в этом тексте: 46
21
Количество пер

## Переразмеченные файлы корпуса находятся в папке texts